In [42]:
import pandas as pd
from shapely.geometry import Point
from geopandas import GeoDataFrame
import requests
import sqlite3

In [43]:
server = "https://geo.api.vlaanderen.be/BWD-IdentifiedBathingWaters/ogc/features/collections/BWDBW/items?f=application%2Fgeo%2Bjson"

In [44]:
# Azure App Gateway blocks Python requests lib by default
response = requests.get(server, headers={
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
})


In [45]:
data = [item['properties'] for item in response.json()['features']]
data

[{'OIDN': 1,
  'UIDN': 1,
  'BWID': 'BEVL_VMM_10',
  'BWNAME': 'De Panne - Leopold',
  'SHORTNAME': 'DPB_Leopold',
  'LONGITUDE': 2.5772,
  'LATITUDE': 51.102,
  'COORDSYS': 'ETRS89',
  'GROUPID': ' ',
  'RBDID': 'BESchelde_VL',
  'RBDNAME': 'Scheldt',
  'RBDSUID': 'BESchelde_VL',
  'RBDSUNAME': 'Scheldt',
  'WBID': 'CWSB1',
  'WBNAME': 'Belgische kust',
  'NWUNITID': ' ',
  'NWUNITNAME': ' ',
  'BWKEY': 'De Panne',
  'BWTYPE': 1,
  'BWATERCAT': 'C',
  'SPECGEOCON': 'N'},
 {'OIDN': 2,
  'UIDN': 2,
  'BWID': 'BEVL_VMM_100',
  'BWNAME': 'Koksijde - Duinpark - Oostduinkerke',
  'SHORTNAME': 'KOK_Duinpark_ODK',
  'LONGITUDE': 2.6796,
  'LATITUDE': 51.1385,
  'COORDSYS': 'ETRS89',
  'GROUPID': ' ',
  'RBDID': 'BESchelde_VL',
  'RBDNAME': 'Scheldt',
  'RBDSUID': 'BESchelde_VL',
  'RBDSUNAME': 'Scheldt',
  'WBID': 'CWSB1',
  'WBNAME': 'Belgische kust',
  'NWUNITID': ' ',
  'NWUNITNAME': ' ',
  'BWKEY': 'Koksijde',
  'BWTYPE': 1,
  'BWATERCAT': 'C',
  'SPECGEOCON': 'N'},
 {'OIDN': 3,
  'UIDN':

In [46]:
encoding = data[0]['COORDSYS']
dataSet = [
    [
        item['BWID'],
        item['BWNAME'],
        item['LATITUDE'],
        item['LONGITUDE'],
        item['BWKEY'],
    ]
    for item in data
]
dataSetDF = pd.DataFrame(dataSet, columns=["id", "name", "lat", "long", "place"])
geometry = [Point(x, y) for x, y in zip(dataSetDF['lat'], dataSetDF['long'])]

In [47]:
gdf = GeoDataFrame(dataSetDF, geometry=geometry).set_index('id')

In [48]:
gdf = gdf.set_crs(4258)

In [49]:
gdf.crs

<Geographic 2D CRS: EPSG:4258>
Name: ETRS89
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: Europe - onshore and offshore: Albania; Andorra; Austria; Belgium; Bosnia and Herzegovina; Bulgaria; Croatia; Cyprus; Czechia; Denmark; Estonia; Faroe Islands; Finland; France; Germany; Gibraltar; Greece; Hungary; Ireland; Italy; Kosovo; Latvia; Liechtenstein; Lithuania; Luxembourg; Malta; Moldova; Monaco; Montenegro; Netherlands; North Macedonia; Norway including Svalbard and Jan Mayen; Poland; Portugal; Romania; San Marino; Serbia; Slovakia; Slovenia; Spain; Sweden; Switzerland; United Kingdom (UK) including Channel Islands and Isle of Man; Vatican City State.
- bounds: (-16.1, 32.88, 40.18, 84.73)
Datum: European Terrestrial Reference System 1989 ensemble
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [52]:
gdf.to_crs(4326)
locations = gdf[[
    'name', 'place', 'lat', 'long'
]].rename(columns={
    "place": "alternate_name"
})
locations

,name,alternate_name,lat,long
id,,,,
BEVL_VMM_10,De Panne - Leopold,De Panne,51.1020,2.5772
BEVL_VMM_100,Koksijde - Duinpark - Oostduinkerke,Koksijde,51.1385,2.6796
BEVL_VMM_107760,Bocholt - Goolderheide,Bocholt,51.1739,5.5354
BEVL_VMM_110,Koksijde - Groenendijk - Oostduinkerke,Koksijde,51.1448,2.6996
BEVL_VMM_115,Nieuwpoort - Leopoldplein,Nieuwpoort,51.1485,2.7080
...,...,...,...,...
BEVL_VMM_848775,Dessel - Campinastrand,Dessel,51.2630,5.1362
BEVL_VMM_848795,Mol - Familiestrand Postel,Mol,51.2778,5.1569
BEVL_VMM_871500,Jabbeke - Klein Strand,Jabbeke,51.1860,3.1014


In [53]:
db = sqlite3.connect("../dataset.sqlite3")
locations.to_sql("locations", db, if_exists="append")


89